In [3]:
# All the imports go here
import cv2
import numpy as np
import mediapipe as mp
from collections import deque


# Giving different arrays to handle color points of different colour
# deque are data structures that we are using
# here we are using blue, green, red and yellow colors, we can use more if we want
# max len is 1024, we can increase or decrease it according to our preferences
bpoints = [deque(maxlen=1024)]
gpoints = [deque(maxlen=1024)]
rpoints = [deque(maxlen=1024)]
ypoints = [deque(maxlen=1024)]


# These indices will be used to mark the points in particular arrays of specific colour
# only the index of selected color will be 1, rest will be 0
blue_index = 0
green_index = 0
red_index = 0
yellow_index = 0

#The kernel to be used for dilation purpose
# dilation operator takes two pieces of data as inputs. The first is the image which is to be dilated. The second is a (usually small) set of coordinate points known as a structuring element (also known as a kernel)
kernel = np.ones((5,5),np.uint8)

# give hex values of colors
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]
colorIndex = 0

# there will be 2 windows, one static (which we will have to create on our own) and one dynamic (live)
# Here is code for Canvas setup

# using numpy zeros, we will set the window size, and we will add 255 to convert it from black to white
# np.zeros: returns a new array of given shape and type, where the element's value as 0
paintWindow = np.zeros((471,636,3)) + 255

# now, let us give our rectangles their dimensions
# cv2.rectangle(image, startpoint, endpoint, color, thickness)
paintWindow = cv2.rectangle(paintWindow, (40,1), (140,65), (0,0,0), 2) # clear (absence of any color on the display)
paintWindow = cv2.rectangle(paintWindow, (160,1), (255,65), (255,0,0), 2) # red
paintWindow = cv2.rectangle(paintWindow, (275,1), (370,65), (0,255,0), 2) # yellow
paintWindow = cv2.rectangle(paintWindow, (390,1), (485,65), (0,0,255), 2) # blue
paintWindow = cv2.rectangle(paintWindow, (505,1), (600,65), (0,255,255), 2) # green

# let us give them their labels and fonts
# cv2.putText will create buttons
# cv2.line(img, pt1, pt2, color[, thickness[, lineType[, shift]]])
# (image, label, location, font, fontsize, bgcolor, line thickness, line type (CV__AA- antialiased line))
cv2.putText(paintWindow, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

# give the name 'paint' to the created window
cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)
# we have successfully created the window now


# initialize mediapipe
mpHands = mp.solutions.hands # initialize
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
# max hands should be 1 in this case otherwise it will also detect another hand if any in the frame
# min detection confidence can be changed according to the resolution of your webcam, it will detect it very easily if we lower it
mpDraw = mp.solutions.drawing_utils # it will detect knuckle points (landmarks)


# Initialize the webcam
cap = cv2.VideoCapture(0) # to start the video instance, values will be 1, 2, 3, etc if you have a webcam
ret = True
while ret:
    # Read each frame from the webcam
    ret, frame = cap.read() # start from here if return var is true

    x, y, c = frame.shape # (x: height, y: width, c: no. of channels in i/p image)

    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # to use both opencv (BGR) and other (RGB) together
    
    # in the camera (live) window, the camera will only show the image and not the rectangles, right? so, we will have to cerate rectangles in all the frames
    # cv2.rectangle(image, startpoint, endpoint, color, thickness)
    frame = cv2.rectangle(frame, (40,1), (140,65), (0,0,0), 2)
    frame = cv2.rectangle(frame, (160,1), (255,65), (255,0,0), 2)
    frame = cv2.rectangle(frame, (275,1), (370,65), (0,255,0), 2)
    frame = cv2.rectangle(frame, (390,1), (485,65), (0,0,255), 2)
    frame = cv2.rectangle(frame, (505,1), (600,65), (0,255,255), 2)
    # (image, label, location, font, fontsize, bgcolor, line thickness, line type (CV__AA- antialiased line))
    cv2.putText(frame, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

    # Get hand landmark prediction
    result = hands.process(framergb) # we will give converted frames to process hands

    # post process the result
    if result.multi_hand_landmarks:
        # there will be no more than 2 hands but if someone is ambidextrous and wants to write with both the hands then they can use this
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark: # lm: landmark
                # adjust the points (0,1) according to their length sizes by multiplying with the ratio
                lmx = int(lm.x * 640)
                lmy = int(lm.y * 480)
                landmarks.append([lmx, lmy]) # it will detect both coordinates of both hands


            # Drawing landmarks on frames
            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS) # draw the landmarks
        fore_finger = (landmarks[8][0],landmarks[8][1]) # take landmark of fore finger
        center = fore_finger # consider the fore finger as your center
        thumb = (landmarks[4][0],landmarks[4][1]) # take landmark of your thumb
        # cv2.circle(image, center coordinates, radius, color, thickness of color line in px (-1: color specified))
        cv2.circle(frame, center, 3, (0,255,0),-1) # create a circle with fore finger as your center and if your thumb is within a particular radius of your finger, it means that it is close to your finger and that your finger is down and you don't want to draw
        print(center[1]-thumb[1]) # we are constantly printing the distance between the finger and the thumb
        
        if (thumb[1]-center[1]<30): # if difference of y coordinates of thumb and finger are less than 30 then do not draw
            # we will still store the points to use them later
            bpoints.append(deque(maxlen=512))
            blue_index += 1
            gpoints.append(deque(maxlen=512))
            green_index += 1
            rpoints.append(deque(maxlen=512))
            red_index += 1
            ypoints.append(deque(maxlen=512))
            yellow_index += 1

        elif center[1] <= 65: # if difference if greater than 65, means that the finger is up on the buttons
            
            if 40 <= center[0] <= 140: # Clear Button between columns 40 and 140
                bpoints = [deque(maxlen=512)]
                gpoints = [deque(maxlen=512)]
                rpoints = [deque(maxlen=512)]
                ypoints = [deque(maxlen=512)]                
                
                # empty all points in the deque when finger points on clear
                blue_index = 0
                green_index = 0
                red_index = 0
                yellow_index = 0
                paintWindow[67:,:,:] = 255
                
                # change the color index to the selected color if it is not on clear
            elif 160 <= center[0] <= 255:
                    colorIndex = 0 # Blue
            elif 275 <= center[0] <= 370:
                    colorIndex = 1 # Green
            elif 390 <= center[0] <= 485:
                    colorIndex = 2 # Red
            elif 505 <= center[0] <= 600:
                    colorIndex = 3 # Yellow
        else :
            # if its not on button and if the finger is not down, we need to draw
            if colorIndex == 0:
                bpoints[blue_index].appendleft(center) # we appended or added the point to the data structure
                # it will draw all the points as your finger and will print them on all the frames and hence will give an illusion that a line has been drawn
            elif colorIndex == 1:
                gpoints[green_index].appendleft(center)
            elif colorIndex == 2:
                rpoints[red_index].appendleft(center)
            elif colorIndex == 3:
                ypoints[yellow_index].appendleft(center)
    # Append the next deques when nothing is detected to avoid messing up
    # like if you are not drawing for now, it will store the last location on which you drew and will start from the new location instead of drawing all the locations in between
    else:
        bpoints.append(deque(maxlen=512))
        blue_index += 1
        gpoints.append(deque(maxlen=512))
        green_index += 1
        rpoints.append(deque(maxlen=512))
        red_index += 1
        ypoints.append(deque(maxlen=512))
        yellow_index += 1

    # Draw lines of all the colors on the canvas and frame
    points = [bpoints, gpoints, rpoints, ypoints] # we took all the points we recorded
    
    # iterate the points and print them on both canvas
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    cv2.imshow("Output", frame)
    cv2.imshow("Paint", paintWindow)

    if cv2.waitKey(1) == ord('q'):
        break # it will break the code when you press 'q'

# release the webcam and destroy all active windows
cap.release()
cv2.destroyAllWindows()

48
47
44
46
37
41
40
43
47
50
48
47
50
49
49
47
52
56
53
53
46
-31
-52
-70
-79
-98
-98
-100
-102
-105
-108
-108
-108
-108
-104
-108
-106
-104
-105
-103
-111
-118
-115
-118
-99
-85
-69
-45
-19
-13
-13
-17
-19
-17
-19
-16
-15
-19
-18
-18
-21
-22
-20
-12
-15
-18
-15
-16
-12
-14
-14
-18
-15
-17
-15
-15
-9
-11
-16
-22
-21
-15
-17
-19
-15
-15
-12
-11
-12
-13
-13
-12
-19
-30
-41
-47
-56
-62
-63
-78
-79
-84
-86
-88
-94
-94
-95
-94
-96
-92
-91
25
17
17
-110
-113
-114
-120
-118
-117
-115
-115
-118
-113
-112
-115
-112
-113
-113
-115
-115
-114
-116
-119
-119
-120
-125
-126
-129
-131
-129
-130
-127
-126
-127
-133
-130
-128
-132
-128
-129
-130
-126
-113
-130
-122
-113
-101
-86
-79
-73
-65
-65
-65
-77
-90
-96
-99
-98
-105
-106
-107
-107
-113
-111
-113
-117
-119
-117
-116
-121
-122
-125
-121
-119
-118
-121
-110
-126
-133
-133
-129
-121
-126
-126
-118
-123
-122
-119
-120
-116
-75
-48
-24
-3
1
-11
-8
-6
-12
-13
-20
-19
-25
-25
-26
-20
-21
-20
-24
-24
-16
-6
-4
-5
-2
-4
-2
-3
-5
-4
-5
-9
-11
-25
-38
-87
